In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import re
import pickle

In [2]:
stock_init = pd.read_csv('stock_name.csv')['ticker']
placeholder = pd.read_csv('placeholder.csv')
placeholder['Date'] = pd.to_datetime(placeholder['Date']).astype(np.datetime64)
placeholder = placeholder.set_index('Date')
stock_init.head()

0       ACG.BK
1       AHC.BK
2       AKR.BK
3    AMARIN.BK
4    AMATAV.BK
Name: ticker, dtype: object

In [3]:
stock_name = stock_init.values.ravel()
stock_name = [re.sub('\.', '_', txt) for txt in stock_name]
stock_name[:5]

['ACG_BK', 'AHC_BK', 'AKR_BK', 'AMARIN_BK', 'AMATAV_BK']

In [4]:
stock_name

['ACG_BK',
 'AHC_BK',
 'AKR_BK',
 'AMARIN_BK',
 'AMATAV_BK',
 'AS_BK',
 'ASAP_BK',
 'AYUD_BK',
 'BFIT_BK',
 'BROCK_BK',
 'BUI_BK',
 'BWG_BK',
 'CCET_BK',
 'CGH_BK',
 'CI_BK',
 'CNT_BK',
 'COL_BK',
 'COTTO_BK',
 'CPL_BK',
 'CSC_BK',
 'CSR_BK',
 'CTW_BK',
 'DTC_BK',
 'EVER_BK',
 'FN_BK',
 'FSS_BK',
 'GC_BK',
 'GJS_BK',
 'GLOCON_BK',
 'GPI_BK',
 'GRAND_BK',
 'ICC_BK',
 'INET_BK',
 'J_BK',
 'KCAR_BK',
 'KDH_BK',
 'KEX_BK',
 'KWC_BK',
 'KYE_BK',
 'LEE_BK',
 'LHFG_BK',
 'LST_BK',
 'MALEE_BK',
 'MANRIN_BK',
 'MBKET_BK',
 'MFC_BK',
 'MICRO_BK',
 'MILL_BK',
 'MPIC_BK',
 'MSC_BK',
 'MTI_BK',
 'NCAP_BK',
 'NFC_BK',
 'NKI_BK',
 'NMG_BK',
 'NOK_BK',
 'NSI_BK',
 'OGC_BK',
 'OISHI_BK',
 'PAF_BK',
 'PAP_BK',
 'PCSGH_BK',
 'PDI_BK',
 'PL_BK',
 'PLE_BK',
 'PORT_BK',
 'POST_BK',
 'PRECHA_BK',
 'PSH_BK',
 'PTL_BK',
 'RICHY_BK',
 'ROCK_BK',
 'ROH_BK',
 'RPC_BK',
 'SCCC_BK',
 'SCN_BK',
 'SFP_BK',
 'SHANG_BK',
 'SIAM_BK',
 'SISB_BK',
 'SKE_BK',
 'SLP_BK',
 'SMIT_BK',
 'SMK_BK',
 'SQ_BK',
 'SSC_BK',
 'SSSC_BK

In [5]:
full_trade_day = 1462
processed_stock_name = []
for stock in stock_name:
    curr_stock = pd.read_csv(os.path.join(os.getcwd(), 'raw', 'price', stock + '.csv'))
    trading_day = len(curr_stock)
    curr_stock['Date'] = pd.to_datetime(curr_stock['Date']).astype(np.datetime64)
    curr_stock = curr_stock.set_index('Date')
    curr_stock = placeholder.join(curr_stock, on='Date', how='left')
    curr_stock = curr_stock.fillna(method='ffill')
    curr_stock['return'] = curr_stock['Close'].diff() / curr_stock['Close'].shift(periods=1)
    curr_stock['log_return'] = np.log(curr_stock['Close'] / curr_stock['Close'].shift(periods=1))
    curr_stock = curr_stock.drop(index=datetime.datetime(2015, 1, 5, 0, 0, 0))
    if sum((curr_stock['return'] == 0)) / len(curr_stock) > 0.8:
        print('Not enough movement: {}'.format(stock))
        continue
    if stock == 'UTP_BK':
        print('Stock split: {}'.format(stock))
        continue
    if trading_day >= 0.98 * full_trade_day:
        curr_stock.to_csv(os.path.join(os.getcwd(), 'processed', 'price', stock+'.csv'))
        processed_stock_name.append(stock)
    else:
        print('Not enough trading days ({} days) for {}'.format(trading_day, stock))

Not enough trading days (369 days) for ACG_BK
Not enough trading days (1229 days) for AMATAV_BK
Not enough trading days (915 days) for ASAP_BK
Not enough movement: BUI_BK
Not enough trading days (587 days) for COTTO_BK
Not enough movement: CSR_BK
Not enough trading days (1007 days) for FN_BK
Not enough trading days (751 days) for GPI_BK
Not enough trading days (1253 days) for J_BK
Not enough trading days (5 days) for KEX_BK
Not enough trading days (60 days) for MICRO_BK
Not enough trading days (35 days) for NCAP_BK
Not enough trading days (757 days) for PORT_BK
Not enough trading days (994 days) for PSH_BK
Not enough trading days (1427 days) for SCN_BK
Not enough movement: SFP_BK
Not enough trading days (507 days) for SISB_BK
Not enough trading days (758 days) for SKE_BK
Not enough trading days (1381 days) for SLP_BK
Not enough trading days (1020 days) for SQ_BK
Not enough trading days (119 days) for STGT_BK
Not enough trading days (0 days) for TGH_BK
Not enough trading days (996 days)

In [6]:
final_stock_name = pd.DataFrame({'tickers': processed_stock_name})
final_stock_name.head()

,tickers
0,AHC_BK
1,AKR_BK
2,AMARIN_BK
3,AS_BK
4,AYUD_BK


In [7]:
final_stock_name.to_csv('tickers.csv', index=False)

In [8]:
with open(os.path.join('processed', 'relation', 'ordered_tickers.pkl'), 'wb') as f:
    pickle.dump(processed_stock_name, f)

In [9]:
len(processed_stock_name)

623